In [1]:
import ee
import geemap

# Authenticate to Earth Engine
try:
  ee.Initialize(project='ee-ana-zonia')
except Exception as e:
  ee.Authenticate()
  ee.Initialize(project='ee-ana-zonia')

from utils import export_image
from utils import map_image

In [3]:
# region of interest is Amazonia
roi = ee.FeatureCollection("projects/ee-ana-zonia/assets/br_biomes").first().geometry()

ecoregions = ee.Image('projects/ee-ana-zonia/assets/ecoregions_br').clip(roi)
mask = ee.Image('projects/ee-ana-zonia/assets/one_hectare_secondary_forest_mask').clip(roi)
ecoregions_masked = ecoregions.updateMask(mask)

In [5]:
## stratify sample points per feature
stratified = ecoregions_masked.stratifiedSample(
    numPoints = 1000,
    classBand = 'ecoreg',
    geometries = True
)

task = ee.batch.Export.table.toDrive(
    collection = stratified,
    description = 'stratified',
    folder = 'datasheets'
)

task.start()

In [20]:
## sample image using stratified points ('random' is the default column name given by randomColumn)
sample = img.sampleRegions(collection=stratified, properties=['ecoregion', 'random'], scale=30)

In [ ]:
frag = ee.Image('projects/ee-ana-zonia/assets/frag_2020')
land_use = LU_sum.addBands([last_LU, last_fire, num_fires, frag, median_mature])

ecoreg = ee.Image("projects/ee-ana-zonia/assets/ecoregions_br")
indig = ee.Image("projects/ee-ana-zonia/assets/indig")
soil = ee.Image("projects/ee-ana-zonia/assets/soil")
protec = ee.Image("projects/ee-ana-zonia/assets/protec")
biomes = ee.Image("projects/ee-ana-zonia/assets/biome_br")
categorical = ecoreg.addBands([indig, soil, protec, biomes])

yearly_si = ee.Image("projects/ee-ana-zonia/assets/yearly_si")
mean_prec = ee.Image("projects/ee-ana-zonia/assets/mean_prec")
cwd_chave = ee.Image("projects/ee-ana-zonia/assets/cwd_chave").clip(roi)
climate = yearly_si.addBands([mean_prec, cwd_chave])

#addBands(ee.Image.pixelLonLat()).